In [1]:
import numpy as np
import pandas as pd 
from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector, TimeDistributed,Masking
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K
import tensorflow as tf

Using TensorFlow backend.
/usr/local/google/home/vere/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/google/home/vere/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/google/home/vere/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/google/ho

In [2]:
import os
print(os.listdir())

['IPL-Functional_LSTM-multiout.ipynb', 'machine-learning-ex7', 'IPL-Sequential-LSTM.ipynb', 'IPL-LSTM-OneToOne_Outputs_padded_start.ipynb', '.ipynb_checkpoints', 'IPL-Sequential_LSTM-multiout.ipynb', 'org.octave.Octave.desktop', 'IPL', 'IPL-LSTM-OneToOne_Outputs.ipynb', 'API requests.ipynb', 'machine-learning-ex6', 'machine-learning-ex1', 'machine-learning-ex8', 'deliveries.csv', 'machine-learning-ex4', 'machine-learning-ex2', 'IPL-Functional_LSTM.ipynb', 'machine-learning-ex5', 'matches.csv', 'machine-learning-ex3']


In [3]:
matches=pd.read_csv('matches.csv')
deliveries=pd.read_csv('deliveries.csv')

# Transform 'player dismissed' from NaNs / a player name, to a binary indicator col indicating wicket
deliveries.player_dismissed=deliveries.player_dismissed.notnull().astype(int)
deliveries.columns


Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batsman', 'non_striker', 'bowler', 'is_super_over', 'wide_runs',
       'bye_runs', 'legbye_runs', 'noball_runs', 'penalty_runs',
       'batsman_runs', 'extra_runs', 'total_runs', 'player_dismissed',
       'dismissal_kind', 'fielder'],
      dtype='object')

In [4]:
# Turn batting / bowling teams column from a string to a one hot feature encoding


deliveries['batting_team'].unique()
A = deliveries['batting_team'].str.get_dummies()
B = deliveries['bowling_team'].str.get_dummies()
deliveries = deliveries.join(A, lsuffix='_l', rsuffix='_r')
deliveries = deliveries.join(B, lsuffix='_batfirst', rsuffix='_bowlfirst')

deliveries.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,Kings XI Punjab_bowlfirst,Kochi Tuskers Kerala_bowlfirst,Kolkata Knight Riders_bowlfirst,Mumbai Indians_bowlfirst,Pune Warriors_bowlfirst,Rajasthan Royals_bowlfirst,Rising Pune Supergiant_bowlfirst,Rising Pune Supergiants_bowlfirst,Royal Challengers Bangalore_bowlfirst,Sunrisers Hyderabad_bowlfirst
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,0,1,0
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,0,1,0
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,0,1,0
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,0,1,0
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,0,1,0


In [5]:
deliveries.columns

# Feature columns with string columns removed
# I.e. names of batsman, non-striker, bowler and any names relating to a wicket are ommited 
# This is because this is only a prototype. 
# Also I'm not sure there are enough examples to avoid overfitting if the number of features becomes too large

feature_cols = ['match_id', 'inning','over', 'ball', 'is_super_over', 'wide_runs',
       'bye_runs', 'legbye_runs', 'noball_runs', 'penalty_runs',
       'batsman_runs', 'extra_runs', 'total_runs', 'player_dismissed', 'Chennai Super Kings_batfirst',
       'Deccan Chargers_batfirst', 'Delhi Capitals_batfirst',
       'Delhi Daredevils_batfirst', 'Gujarat Lions_batfirst',
       'Kings XI Punjab_batfirst', 'Kochi Tuskers Kerala_batfirst',
       'Kolkata Knight Riders_batfirst', 'Mumbai Indians_batfirst',
       'Pune Warriors_batfirst', 'Rajasthan Royals_batfirst',
       'Rising Pune Supergiant_batfirst', 'Rising Pune Supergiants_batfirst',
       'Royal Challengers Bangalore_batfirst', 'Sunrisers Hyderabad_batfirst',
       'Chennai Super Kings_bowlfirst', 'Deccan Chargers_bowlfirst',
       'Delhi Capitals_bowlfirst', 'Delhi Daredevils_bowlfirst',
       'Gujarat Lions_bowlfirst', 'Kings XI Punjab_bowlfirst',
       'Kochi Tuskers Kerala_bowlfirst', 'Kolkata Knight Riders_bowlfirst',
       'Mumbai Indians_bowlfirst', 'Pune Warriors_bowlfirst',
       'Rajasthan Royals_bowlfirst', 'Rising Pune Supergiant_bowlfirst',
       'Rising Pune Supergiants_bowlfirst',
       'Royal Challengers Bangalore_bowlfirst',
       'Sunrisers Hyderabad_bowlfirst']

In [6]:
# Number of hidden states in LSTM, could also be called n_j
n_a=64

# Number of features
feature_size = len(feature_cols)

# number of examples
m = len(deliveries.match_id.unique())

# List of all the match id's used in the dataset (note not in temporal order and some ids are quite random)
match_ids = deliveries.match_id.unique()

# Number of deliveries for each match
Number_of_deliveries = [deliveries.match_id.value_counts(sort=False).to_dict()[x] for x in match_ids]

# Longest game, number of deliveries
max_game_length = max(Number_of_deliveries)

# find the indices of the match starts in the original dataset
ind_start_of_match = np.insert(np.cumsum(Number_of_deliveries),0,0)
idx = ind_start_of_match



# Turn pandas df into a np array and select features of interest
npdevs = np.asarray(deliveries[feature_cols])

list_of_games = [npdevs[start: end+1,:] for start, end in zip(idx, idx[1:])]
# print(*list_of_games, sep='\n'*2)

# Set up global NN layers with shareable weights
LSTM_cell = LSTM(n_a, return_state = True)  
reshapor = Reshape((1, feature_size)) 
densor = Dense(1,activation='sigmoid')

In [7]:
from keras.preprocessing.sequence import pad_sequences
mask_value = -1000
padded_games = pad_sequences(list_of_games, maxlen=max_game_length, dtype='int32', padding='pre', truncating='pre', value=mask_value)
padded_games

array([[[-1000, -1000, -1000, ..., -1000, -1000, -1000],
        [-1000, -1000, -1000, ..., -1000, -1000, -1000],
        [-1000, -1000, -1000, ..., -1000, -1000, -1000],
        ...,
        [    1,     2,    20, ...,     0,     0,     1],
        [    1,     2,    20, ...,     0,     0,     1],
        [    2,     1,     1, ...,     0,     0,     0]],

       [[-1000, -1000, -1000, ..., -1000, -1000, -1000],
        [-1000, -1000, -1000, ..., -1000, -1000, -1000],
        [-1000, -1000, -1000, ..., -1000, -1000, -1000],
        ...,
        [    2,     2,    20, ...,     0,     0,     0],
        [    2,     2,    20, ...,     0,     0,     0],
        [    3,     1,     1, ...,     0,     0,     0]],

       [[-1000, -1000, -1000, ..., -1000, -1000, -1000],
        [-1000, -1000, -1000, ..., -1000, -1000, -1000],
        [-1000, -1000, -1000, ..., -1000, -1000, -1000],
        ...,
        [    3,     2,    15, ...,     0,     0,     0],
        [    3,     2,    15, ...,     0,    

In [8]:


# Convert to tensor (not sure this is needed anymore)
# and remove the last index (which refers to the end of the last game)
# ind_start_of_match_pad=tf.convert_to_tensor(ind_start_of_match_pad.astype(int)[:-1])


In [9]:
# X shape is m by game length by feature size

def IPL_model(LSTM_units,dropout):
    """
    
    Arguments:


    Returns:
    model -- a model instance in Keras
    """

    deliveries = Input(shape= (max_game_length,feature_size))
    mask = Masking(mask_value=mask_value)(deliveries)
    
    # Propagate the input through an LSTM layer with 'LSTM_units'-dimensional hidden state
    # The returned output should be a batch of sequences.
    X = LSTM(units=LSTM_units,return_sequences=True)(mask)
#     # Add dropout with a probability of 'dropout'
#     X = Dropout(rate=dropout)(X)
#     # Propagate X trough another LSTM layer with 'LSTM_units'-dimensional hidden state
#     # The returned output should be a batch of sequences.
#     X = LSTM(units=LSTM_units, return_sequences=True)(X)
#     # Add dropout with a probability of 'dropout'
#     X = Dropout(rate=dropout)(X)
    # Propagate X through a Dense layer with 1 unit
    X = TimeDistributed(Dense(units=1,activation='sigmoid'))(X)

    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=deliveries,outputs=X)
    

    
    return model

In [10]:
padded_games.shape

(756, 267, 44)

In [11]:
LSTM_units = 128
dropout =0.2
model = IPL_model(LSTM_units,dropout)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [12]:
opt = Adam(lr=0.02, beta_1=0.9, beta_2=0.999, decay=0.01)

model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
a0 = np.zeros((m, n_a))
c0 = np.zeros((m, n_a))

# Don't care how much the team wins or loses by, 
# if the score in this col is greater than 1, then the bat first team won

# Labels are set for every ball of the game i.e. Y is shape max_game_length x number_of_games (m)

Y_final = np.array(matches.win_by_runs.clip(upper=1))
Y = Y_final.reshape(m,1)*np.ones((m,max_game_length))
Y = Y.reshape(m,max_game_length,1)
# X = np.transpose(padded_games.reshape((max_game_length,m,feature_size),order='F' ),(1, 0, 2))
X = padded_games
# X shape is m by game length by feature size
X_train = X[5:]
Y_train = Y[5:]

In [14]:
model.summary()
print("Inputs: "+str(model.input_shape))
print("Outputs:"+str(model.output_shape))
print("Actual input:"+str(X.shape))
print("Actual output:"+str(Y.shape))

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 267, 44)           0         
_________________________________________________________________
masking_1 (Masking)          (None, 267, 44)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 267, 128)          88576     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 267, 1)            129       
Total params: 88,705
Trainable params: 88,705
Non-trainable params: 0
_________________________________________________________________
Inputs: (None, 267, 44)
Outputs:(None, 267, 1)
Actual input:(756, 267, 44)
Actual output:(756, 267, 1)


In [15]:
# Note that poor val_accuracy here isn't necessarily bad news
# The performance over the whole game is averaged out so good performance
# towards the end of a game could be swamped by uncertainty at the start
model.fit(X, Y, verbose = 2, epochs=100, validation_split=0.1,shuffle=True)


Train on 680 samples, validate on 76 samples
Epoch 1/100
 - 10s - loss: 0.6416 - accuracy: 0.5078 - val_loss: 0.6214 - val_accuracy: 0.6068
Epoch 2/100
 - 8s - loss: 0.6211 - accuracy: 0.5320 - val_loss: 0.6174 - val_accuracy: 0.6068
Epoch 3/100
 - 8s - loss: 0.6164 - accuracy: 0.4913 - val_loss: 0.6115 - val_accuracy: 0.6068
Epoch 4/100
 - 8s - loss: 0.6144 - accuracy: 0.5373 - val_loss: 0.6094 - val_accuracy: 0.6068
Epoch 5/100
 - 8s - loss: 0.6152 - accuracy: 0.5398 - val_loss: 0.6098 - val_accuracy: 0.6068
Epoch 6/100
 - 8s - loss: 0.6144 - accuracy: 0.5313 - val_loss: 0.6089 - val_accuracy: 0.6068
Epoch 7/100
 - 8s - loss: 0.6129 - accuracy: 0.5387 - val_loss: 0.6070 - val_accuracy: 0.6068
Epoch 8/100
 - 8s - loss: 0.6126 - accuracy: 0.5432 - val_loss: 0.6091 - val_accuracy: 0.6068
Epoch 9/100
 - 8s - loss: 0.6098 - accuracy: 0.5355 - val_loss: 0.6113 - val_accuracy: 0.6068
Epoch 10/100
 - 8s - loss: 0.6070 - accuracy: 0.5447 - val_loss: 0.6100 - val_accuracy: 0.6068
Epoch 11/100

Epoch 85/100
 - 8s - loss: 0.5798 - accuracy: 0.5692 - val_loss: 0.6172 - val_accuracy: 0.6068
Epoch 86/100
 - 8s - loss: 0.5785 - accuracy: 0.5673 - val_loss: 0.6169 - val_accuracy: 0.6068
Epoch 87/100
 - 8s - loss: 0.5787 - accuracy: 0.5686 - val_loss: 0.6169 - val_accuracy: 0.6068
Epoch 88/100
 - 8s - loss: 0.5843 - accuracy: 0.5668 - val_loss: 0.6070 - val_accuracy: 0.6068
Epoch 89/100
 - 8s - loss: 0.5785 - accuracy: 0.5689 - val_loss: 0.6068 - val_accuracy: 0.6068
Epoch 90/100
 - 8s - loss: 0.5796 - accuracy: 0.5692 - val_loss: 0.6060 - val_accuracy: 0.6068
Epoch 91/100
 - 8s - loss: 0.5783 - accuracy: 0.5684 - val_loss: 0.6069 - val_accuracy: 0.6068
Epoch 92/100
 - 8s - loss: 0.5772 - accuracy: 0.5670 - val_loss: 0.6077 - val_accuracy: 0.6068
Epoch 93/100
 - 8s - loss: 0.5751 - accuracy: 0.5710 - val_loss: 0.6076 - val_accuracy: 0.6068
Epoch 94/100
 - 8s - loss: 0.5733 - accuracy: 0.5778 - val_loss: 0.6078 - val_accuracy: 0.6068
Epoch 95/100
 - 8s - loss: 0.5729 - accuracy: 0.57

In [17]:

# score, acc = model.evaluate(x_test, y_test)
# y_predict = model.predict(X[:,:,:])
# count=0
# for i in range(m):
#     y_diff = y_predict[i,max_game_length-1] -Y[i,max_game_length-1,0]
# #     print(y_predict)
#     if np.linalg.norm(y_diff) > 0.1:
#         count +=1
    
y_predict = model.predict(X[0:6,:,:])
y_predict - Y[0:6]

array([[[-5.04552603e-01],
        [-5.04552603e-01],
        [-5.04552603e-01],
        ...,
        [-1.02857351e-02],
        [-9.91129875e-03],
        [-8.12977552e-03]],

       [[ 4.95447397e-01],
        [ 4.95447397e-01],
        [ 4.95447397e-01],
        ...,
        [ 1.85012817e-04],
        [ 1.89006329e-04],
        [ 9.00328159e-05]],

       [[ 4.95447397e-01],
        [ 4.95447397e-01],
        [ 4.95447397e-01],
        ...,
        [ 1.18672848e-04],
        [ 1.19984150e-04],
        [ 1.51783228e-04]],

       [[ 4.95447397e-01],
        [ 4.95447397e-01],
        [ 4.95447397e-01],
        ...,
        [ 2.42263079e-04],
        [ 2.39700079e-04],
        [ 1.95115805e-04]],

       [[-5.04552603e-01],
        [-5.04552603e-01],
        [-5.04552603e-01],
        ...,
        [-1.02834702e-02],
        [-1.02771521e-02],
        [-8.96275043e-03]],

       [[ 4.95447397e-01],
        [ 4.95447397e-01],
        [ 4.95447397e-01],
        ...,
        [ 1.35898590e